In [ ]:
import matplotlib.pyplot as plt
from Genetic.Representation import Function
from Genetic.NeuroEvolution import NeuroEvolution
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.losses import mse
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import seaborn as sns
from IPython.display import display
import numpy as np
from Model.model import SurvModel, FlchainModel, NwtcoModel, SupportModel
from Data.dataset import Flchain, Support, Nwtco
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import f_oneway, ttest_rel

from Utils.utils import Experiment, FunctionHelper, StatsHelper

In [ ]:
surv_reg1_func = [1, 9, 9, 1, 5, 12, 7, 9, 7]
surv_reg2_func = [1, 9, 12, 1, 5, 12, 7, 9, 7]

surv_reg1_f = FunctionHelper(surv_reg1_func)
surv_reg2_f = FunctionHelper(surv_reg2_func)

surv_reg1_f.plot_surv_loss()
surv_reg2_f.plot_surv_loss()

In [ ]:
model_class = SupportModel
ds = Support('Data/support2.csv')

In [ ]:
(x_train, ye_train, y_train, e_train,
 x_val, ye_val, y_val, e_val,
 x_test, ye_test, y_test, e_test) = ds.get_train_val_test_from_splits(test_id=0, val_id=1)
xy = (x_train, ye_train), (x_val, ye_val), (x_test, ye_test)

In [ ]:
model = model_class(xy, f_loss=surv_reg2_f.f, f_step=None, use_clb=0, sample_size=-1)
model.create_model()

batchSize = 128
epochs = 2
optimizer = Adam(lr=0.0001)
verbose = 1

model.set_config(optimizer, batch_size=batchSize, epochs=epochs, test_mode=True, verbose=verbose)

model.fit()

In [ ]:
model.model.summary()

In [ ]:
logs = []
for i in range(10):
    log, population = NeuroEvolution.SAGA_V0(model,
                                          populationSize=10, 
                                          reductionRate=1,
                                          pateince=2,
                                          tolerance=0,
                                          d=False,
                                          epochs=128,
                                          maxNoChange=np.inf,
                                          initilizationMax=2,
                                          verbose=1,
                                          include=False,
                                          optimizationTask='loss',
                                          evaluation='validation',
                                          timeout=np.inf)
    logs.append(log)


In [ ]:
for log in logs:
    display(log)

In [ ]:
for log in logs:
    best_func_rep = log.iloc[-1]['best_solution']
    best_f = FunctionHelper(best_func_rep)
    best_f.plot_surv_loss(x_min=-1, x_max=1, y_min=-2, y_max=2, num_points=1000)

In [ ]:
exps_cv = []
for i, log in enumerate(logs):
    print(i, '===============================================================================')
    best_func_rep = log.iloc[-1]['best_solution']
    best_f = FunctionHelper(best_func_rep)
    exp = Experiment(model_class=model_class, 
                      ds=ds, 
                      loss_f=best_f.f, 
                      step_f=None, 
                      use_clb=False, 
                      batch_size=128, 
                      epochs=500, 
                      patience=100, 
                      verbose=False)
    exp.get_results_cv()
    exps_cv.append(exp)

In [ ]:
exp4 = Experiment(model_class=model_class, 
                  ds=ds, 
                  loss_f=surv_reg2_f.f, 
                  step_f=None, 
                  use_clb=False, 
                  batch_size=128, 
                  epochs=500, 
                  patience=100, 
                  verbose=False)
exp4.get_results_cv()

In [ ]:
for exp in exps_cv:
    StatsHelper.plot_box(folds1=exp.folds_scores, name1='SurvEvo_Support', folds2=exp4.folds_scores, name2='Surv_reg2')

In [ ]:
for i, log in enumerate(logs):
    best_func_rep = log.iloc[-1]['best_solution']
    print(best_func_rep)
    print(exps[i].folds_scores)
    StatsHelper.get_confidence(exps[i].folds_scores)
    print(exps_cv[i].folds_scores)
    StatsHelper.get_confidence(exps_cv[i].folds_scores)
    print('======================================================================')
    